In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt
import plotly.graph_objects as go
import plotly_express as px
import geopy
from geopy.extra.rate_limiter import  RateLimiter
import geopandas
import folium
from folium.plugins import MarkerCluster
from streamlit_folium import folium_static
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from time import sleep

In [2]:
def get_film_data(title):
    if(not title):
        return None

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get("https://www.imdb.com/")

    search_box = (driver
                  .find_element(By.CSS_SELECTOR, "div.react-autosuggest__container")
                  .find_element(By.TAG_NAME, "input"))
    search_box.send_keys(title)
    sleep(2)

    first_result = driver.find_element(By.CSS_SELECTOR, "a.sc-bqyKva.ehfErK.searchResult.searchResult--const")
    first_result.click()
    sleep(2)

    film_url = driver.current_url

    director = driver.find_element(By.CSS_SELECTOR, "div.ipc-metadata-list-item__content-container")
    director_name = director.find_element(By.TAG_NAME, "a")

    cast_page = (driver
                 .find_element(By.CSS_SELECTOR, "div.ipc-title__wrapper")
                 .find_element(By.TAG_NAME, "a"))
    cast_page.click()
    sleep(1)

    cast_table = driver.find_element(By.CSS_SELECTOR, "table.cast_list").find_element(By.TAG_NAME, "tbody")
    cast_list = cast_table.find_elements(By.TAG_NAME, "tr")

    actors_names = []
    for actor in cast_list[1:]:
        el_class = actor.get_attribute("class")
        if (el_class != "odd" and el_class != "even"):
            break
        actor_name = actor.find_elements(By.TAG_NAME, "td")[1].find_element(By.TAG_NAME, "a").text
        actors_names.append(actor_name)

    return [actors_names, film_url]

def get_locations(film_url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(film_url)
    sleep(1)

    details = driver.find_element(By.CSS_SELECTOR, '[data-testid="title-details-section"]')
    filming_locations = details.find_element(By.CSS_SELECTOR, '[data-testid="title-details-filminglocations"]')
    loc = filming_locations.find_element(By.CSS_SELECTOR,
                                         "a.ipc-metadata-list-item__label.ipc-metadata-list-item__label--link")
    loc.click()
    loc = driver.find_element(By.ID, "filming_locations")
    pr = loc.find_elements(By.TAG_NAME, "div")
    all_locations = []
    for el in pr:
        el_class = el.get_attribute("class")
        if (el_class != "soda sodavote odd" and el_class != "soda sodavote even"):
            continue
        all_locations.append(el.find_element(By.TAG_NAME, "dt").find_element(By.TAG_NAME, "a").text)

    locator = geopy.Nominatim(user_agent="myGeocoder")

    df = pd.DataFrame({"address": all_locations})
    geocode = RateLimiter(locator.geocode, min_delay_seconds=0.01)
    df['location'] = df['address'].apply(geocode)
    df = df.dropna()
    df['point'] = df['location'].apply(lambda cr: tuple(cr.point))
    df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

    coor = df.loc[:, ["address", "latitude", "longitude"]]
    coor.loc[:, "coordinates"] = list(zip(coor["latitude"], coor["longitude"]))

    return [list(coor["address"]), list(coor["coordinates"])]


def draw_locations(coordinates, all_locations):

    map = folium.Map(location=[40.76791227224293, -73.98658282967192], zoom_start=3)
    marker_cluster = MarkerCluster().add_to(map)

    pos = 0
    for point in coordinates:
        folium.Marker(point, popup=all_locations[pos], tooltip=all_locations[pos]).add_to(marker_cluster)
        pos += 1

    folium_static(map)

In [ ]:
title = "The Dark Knight"

if(title):
    res = get_film_data(title)
    actors_names = res[0]
    film_url = res[1]
    loc_info = get_locations(film_url)
    all_locations = loc_info[0]
    coordinates = loc_info[1]

with open('TheDarkKnightLoc.txt', 'w') as f:
    f.write('\n'.join(all_locations))

with open('TheDarkKnightCoor.txt', 'w') as f:
    f.write('\n'.join(str(x) for x in coordinates))

with open('TheDarkKnightActors.txt', 'w') as f:
    f.write('\n'.join(actors_names))

In [34]:
import requests
import pandas as pd
import seaborn as sns
import plotly_express as px

cookies = {
        'uu': 'eyJpZCI6InV1MTQ0OWM1ZDI1ZWE3NGMwMzg2OWQiLCJwcmVmZXJlbmNlcyI6eyJmaW5kX2luY2x1ZGVfYWR1bHQiOmZhbHNlfX0=',
        'ubid-main': '132-0172990-2453204',
        'session-id': '140-0676346-1587024',
        'adblk': 'adblk_no',
        'session-id-time': '2082787201l',
        'session-token': 'm5QQdqTyS46QzwOvxBabaVn1WH4nXGECR+DWAMfcxq4nX+3Q+EaixprdyKv79hoBsskn6iZB8f/Cmxwfnpz/qGAEJLG/9KC7yShPAQc/U5wu5hnPfgEHb+ZmKUD72FaWUJbtXnyZjipSgnzYzZJt1c5h9f3U5CrhDtGDA04HFDFBqvXyrJiy+qM0If/B5iGEHPglVTlwf2ZD8YdJ/W4SDg',
    }

headers = {
    'authority': 'api.graphql.imdb.com',
    'accept': '*/*',
    'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'content-type': 'application/json',
    # Requests sorts cookies= alphabetically
    # 'cookie': 'uu=eyJpZCI6InV1MTQ0OWM1ZDI1ZWE3NGMwMzg2OWQiLCJwcmVmZXJlbmNlcyI6eyJmaW5kX2luY2x1ZGVfYWR1bHQiOmZhbHNlfX0=; ubid-main=132-0172990-2453204; session-id=140-0676346-1587024; adblk=adblk_no; session-id-time=2082787201l; session-token=m5QQdqTyS46QzwOvxBabaVn1WH4nXGECR+DWAMfcxq4nX+3Q+EaixprdyKv79hoBsskn6iZB8f/Cmxwfnpz/qGAEJLG/9KC7yShPAQc/U5wu5hnPfgEHb+ZmKUD72FaWUJbtXnyZjipSgnzYzZJt1c5h9f3U5CrhDtGDA04HFDFBqvXyrJiy+qM0If/B5iGEHPglVTlwf2ZD8YdJ/W4SDg',
    'origin': 'https://www.imdb.com',
    'referer': 'https://www.imdb.com/',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36',
    'x-amzn-sessionid': '140-0676346-1587024',
    'x-imdb-client-name': 'imdb-web-next',
    'x-imdb-user-country': 'RU',
    'x-imdb-user-language': 'ru-RU',
}

params = {
    'operationName': 'BatchPage_HomeMain',
    'variables': '{"fanPicksFirst":30,"topPicksFirst":30}',
    'extensions': '{"persistedQuery":{"sha256Hash":"43d7cd8f063ac6cb0784c3accec54e7fdb2fb95fcd83f63fc7022cc39f6acf85","version":1}}',
}

response = requests.get('https://api.graphql.imdb.com/', params=params, cookies=cookies, headers=headers)
#END FROM

found = response.json()
fan_list = found["data"]["fanPicksTitles"]["edges"]
movie_list = []
for el in fan_list:
    genres = []
    for el1 in el["node"]["titleCardGenres"]["genres"]:
        genres.append(el1["text"])
    
    movie_list.append([el["node"]["originalTitleText"]["text"],genres])
    
fan_fav = pd.DataFrame(movie_list,columns = ["movie", "genres"])
dct_count = {}
for ind,row in fan_fav.iterrows():
    for el in row["genres"]:
        if(el in dct_count):
            dct_count[el] += 1
        else:
            dct_count[el] = 1
    
res = pd.DataFrame(list(dct_count.items()), columns = ["genre", "num"])
px.histogram(res, x = "genre", y = "num")

In [36]:
px.data.tips()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [38]:
dialogues = pd.read_csv("joker.csv")
dialogues
s = ""
for ind,row in dialogues.iterrows():
    s = s + row["line"]
s

' (while driving) Three of a kind, let\'s do this. That\'s it? Three guys? Two guys on the roof. Every guy gets a share. Five shares is plenty. Six shares. Don\'t forget the guy who planned the job. (scoffing) He thinks he can sit it out and still take a slice. I know why they call him the Joker. So, why do they call him the Joker? I heard he wears makeup. Makeup? Yeah, to scare people. Y\'know, war paint. (taking the lead) All right, everybody, hands up, heads down! I said, hands up, heads down! (pulling one of the employees over the desk) Let\'s go, pal, I\'m making a withdrawal here. (running to another employee) I said, hands up! Here comes the silent alarm. And there it goes. Heads down! (dragging the other employee onto the floor) All right, tootsie, you\'re taking a dive with me. Down! I said, stay down there! Funny, it didn\'t dial 911. It was trying to reach a private number. Is it a problem? No, I\'m done here. Sit down! Down! Down! I said, stay down there! Obviously, we don\